In [ ]:
import matplotlib.pyplot as plt

# Define a simple class to hold request metrics
class RequestMetrics:
    def __init__(self, arrival_time, last_token_time, first_scheduled_time, first_token_time, time_in_queue, finished_time, scheduler_time, model_forward_time, model_execute_time, spec_token_acceptance_counts):
        self.arrival_time = arrival_time
        self.last_token_time = last_token_time
        self.first_scheduled_time = first_scheduled_time
        self.first_token_time = first_token_time
        self.time_in_queue = time_in_queue
        self.finished_time = finished_time
        self.scheduler_time = scheduler_time
        self.model_forward_time = model_forward_time
        self.model_execute_time = model_execute_time
        self.spec_token_acceptance_counts = spec_token_acceptance_counts

# List of all 32 requests based on provided data
requests = [
    RequestMetrics(1752222573.5752606, 1752222596.4310427, 1752222573.6119235, 1752222573.9762976, 0.03666281700134277, 1752222596.4313383, 0.10524989900204673, None, None, [0]),
    RequestMetrics(1752222573.576714, 1752222612.1901927, 1752222573.6119235, 1752222573.9762976, 0.03520941734313965, 1752222612.1904886, 0.10872782199930953, None, None, [0]),
    RequestMetrics(1752222573.5778427, 1752222631.5831342, 1752222596.585242, 1752222596.7704713, 23.007399320602417, 1752222631.5833745, 0.11060420599733334, None, None, [0]),
    RequestMetrics(1752222573.5789623, 1752222650.805597, 1752222612.3347464, 1752222612.5214827, 38.755784034729004, 1752222650.8058217, 0.11015307299385313, None, None, [0]),
    RequestMetrics(1752222573.5800383, 1752222672.858666, 1752222631.7333508, 1752222631.9233625, 58.15331244468689, 1752222672.8589199, 0.11167805800641872, None, None, [0]),
    RequestMetrics(1752222573.5811367, 1752222693.5305297, 1752222650.9558172, 1752222651.170655, 77.374680519104, 1752222693.5307927, 0.11164938299179994, None, None, [0]),
    RequestMetrics(1752222573.5822372, 1752222716.7306612, 1752222673.0111227, 1752222673.2013767, 99.4288854598999, 1752222716.7309349, 0.11193481500322378, None, None, [0]),
    RequestMetrics(1752222573.5833962, 1752222738.3911505, 1752222693.684632, 1752222693.8762252, 120.10123586654663, 1752222738.3913972, 0.11345151100476869, None, None, [0]),
    RequestMetrics(1752222573.584508, 1752222760.5815423, 1752222716.8856905, 1752222717.0765595, 143.30118250846863, 1752222760.581775, 0.11261772299349104, None, None, [0]),
    RequestMetrics(1752222573.5856485, 1752222782.7193744, 1752222738.5470765, 1752222738.7348387, 164.9614279270172, 1752222782.7196226, 0.11262863800175182, None, None, [0]),
    RequestMetrics(1752222573.5867808, 1752222805.4884562, 1752222760.7330678, 1752222760.940114, 187.1462869644165, 1752222805.4886875, 0.11148067601516232, None, None, [0]),
    RequestMetrics(1752222573.587871, 1752222827.982756, 1752222782.8737438, 1752222783.1210303, 209.2858726978302, 1752222827.9830012, 0.1132250270111399, None, None, [0]),
    RequestMetrics(1752222573.588955, 1752222851.2470603, 1752222805.6405363, 1752222805.8904417, 232.05158138275146, 1752222851.2473142, 0.11264819000507487, None, None, [0]),
    RequestMetrics(1752222573.590009, 1752222873.5545695, 1752222828.1380877, 1752222828.3292265, 254.54807877540588, 1752222873.554826, 0.11175929900036863, None, None, [0]),
    RequestMetrics(1752222573.5911405, 1752222896.514896, 1752222851.4004421, 1752222851.5887823, 277.80930161476135, 1752222896.5151443, 0.1125393910110688, None, None, [0]),
    RequestMetrics(1752222573.5922296, 1752222918.282544, 1752222873.7114513, 1752222873.9030516, 300.1192216873169, 1752222918.2827868, 0.11165372301320531, None, None, [0]),
    RequestMetrics(1752222573.5933456, 1752222941.6807044, 1752222896.6680722, 1752222896.9083803, 323.0747265815735, 1752222941.6809614, 0.11271911899711995, None, None, [0]),
    RequestMetrics(1752222573.5944226, 1752222964.081719, 1752222918.4367347, 1752222918.6298938, 344.84231209754944, 1752222964.0819428, 0.11374511699295908, None, None, [0]),
    RequestMetrics(1752222573.5955553, 1752222986.9538484, 1752222941.8355272, 1752222942.078021, 368.2399718761444, 1752222986.9541054, 0.11266092799860417, None, None, [0]),
    RequestMetrics(1752222573.596634, 1752223009.5827, 1752222964.237189, 1752222964.4279778, 390.6405551433563, 1752223009.5829384, 0.11334874600288458, None, None, [0]),
    RequestMetrics(1752222573.5976768, 1752223032.7560565, 1752222987.1060483, 1752222987.3480647, 413.508371591568, 1752223032.7563024, 0.11253265600225859, None, None, [0]),
    RequestMetrics(1752222573.5988002, 1752223055.4160697, 1752223009.7408252, 1752223009.9843981, 436.1420249938965, 1752223055.416302, 0.11264994500379544, None, None, [0]),
    RequestMetrics(1752222573.599944, 1752223078.2495182, 1752223032.9073205, 1752223033.1281857, 459.3073763847351, 1752223078.2497575, 0.11277828300035253, None, None, [0]),
    RequestMetrics(1752222573.6013153, 1752223100.9349437, 1752223055.5713875, 1752223055.828883, 481.9700722694397, 1752223100.9351761, 0.11409180000646302, None, None, [0]),
    RequestMetrics(1752222573.6024272, 1752223124.4419045, 1752223078.402052, 1752223078.657628, 504.7996246814728, 1752223124.442154, 0.11385910700300883, None, None, [0]),
    RequestMetrics(1752222573.6035712, 1752223147.058592, 1752223101.091098, 1752223101.314281, 527.4875268936157, 1752223147.0588863, 0.11280050399591346, None, None, [0]),
    RequestMetrics(1752222573.604634, 1752223169.4740534, 1752223124.594437, 1752223124.7854676, 550.9898028373718, 1752223169.4743028, 0.11199166400228933, None, None, [0]),
    RequestMetrics(1752222573.605698, 1752223191.6757214, 1752223147.2151716, 1752223147.4060705, 573.6094734668732, 1752223191.6759539, 0.11249751799641672, None, None, [0]),
    RequestMetrics(1752222573.6068199, 1752223214.482076, 1752223169.6261463, 1752223169.826994, 596.0193264484406, 1752223214.4823222, 0.11351430299464482, None, None, [0]),
    RequestMetrics(1752222573.6079664, 1752223237.3572555, 1752223191.8338723, 1752223192.0445263, 618.2259058952332, 1752223237.3575084, 0.11449833599908743, None, None, [0]),
    RequestMetrics(1752222573.6090539, 1752223259.539771, 1752223214.6341343, 1752223214.8984437, 641.0250804424286, 1752223259.5400193, 0.11063414799855309, None, None, [0]),
    RequestMetrics(1752222573.610123, 1752223281.2130013, 1752223237.5117261, 1752223237.7072113, 663.9016032218933, 1752223281.2132602, 0.08522950999577006, None, None, [0])
]

# Find the earliest arrival time as reference point
t0 = min([req.arrival_time for req in requests])

# Create the Gantt chart
fig, ax = plt.subplots(figsize=(15, 10))

for i, req in enumerate(requests):
    # Calculate relative times for each phase
    queue_start = req.arrival_time - t0
    queue_duration = req.first_scheduled_time - req.arrival_time
    prep_start = req.first_scheduled_time - t0
    prep_duration = req.first_token_time - req.first_scheduled_time
    proc_start = req.first_token_time - t0
    proc_duration = req.finished_time - req.first_token_time
    
    # Plot bars for queueing, preparation, and processing phases
    ax.barh(i, queue_duration, left=queue_start, color='gray', label='Queueing' if i == 0 else "")
    ax.barh(i, prep_duration, left=prep_start, color='blue', label='Preparation' if i == 0 else "")
    ax.barh(i, proc_duration, left=proc_start, color='green', label='Processing' if i == 0 else "")

# Customize the plot
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Request ID')
ax.set_yticks(range(len(requests)))
ax.set_yticklabels([f'Req {i+1}' for i in range(len(requests))])
ax.set_title('vLLM Request Handling Timeline')
ax.set_xlim(0, 750)  # Set x-axis limit to cover the entire duration (~707 seconds)
ax.legend()
plt.tight_layout()
plt.show()